# Stigma against Opioid Use Disorder varies by Personal Use status

```{margin} 
**To follow the full analysis, click through the hidden analysis code below**
```

In [ ]:
# import packages
import os
import pandas as pd
import numpy as np
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import bootstrap
import plotly_express as px

pd.set_option('mode.chained_assignment', None)

In [ ]:
# import data and metadata (data dictionaries)
os.chdir("P:/3645/Common/Protocol 2 Custom Survey/Analysis/Data File/")

df, meta = pyreadstat.read_sav('./3645_JCOIN_HEAL Initiative 2021_NORC_Jan2022_1.sav',apply_value_formats=True)

## import data
#df1 = pd.read_csv("jcoin_norc_omnibus_survey1_feb2020_072821-data.tsv", sep='\t')
#df2 = pd.read_csv("jcoin_norc_omnibus_survey2_april2020_072821-data.tsv", sep='\t')
#df3 = pd.read_csv("jcoin_norc_omnibus_survey3_june2020_072821-data.tsv", sep='\t')
#df4 = pd.read_csv("jcoin_norc_omnibus_survey4_oct2020_072821-data.tsv", sep='\t')

## import metadata (data dictionaries)
#meta1 = pd.read_csv("jcoin_norc_omnibus_survey1_feb2020_072821-data_dictionary.tsv", sep='\t')
#meta2 = pd.read_csv("jcoin_norc_omnibus_survey2_april2020_072821-data_dictionary.tsv", sep='\t')
#meta3 = pd.read_csv("jcoin_norc_omnibus_survey3_june2020_072821-data_dictionary.tsv", sep='\t')
#meta4 = pd.read_csv("jcoin_norc_omnibus_survey4_oct2020_072821-data_dictionary.tsv", sep='\t')

In [ ]:
df.head()

In [ ]:
# narrow down the dataset to only variables that are collected at each of the time-points

# standardize column names across datasets and metadatasets
for df in [df]:
    df.columns = df.columns.str.lower()
    
#for meta in [meta]:
#    meta.name = meta.name.str.lower()  
    
## combine data from all surveys into one long dataset, keeping only vars available across all survey datasets (time-points)
#all_df = pd.concat([df1, df2, df3, df4], keys=['feb2020','apr2020','jun2020','oct2020'], join="inner").reset_index().rename(columns={"level_0": "time-point"})

## combine metadata (data dictionaries) from all surveys into one long metadataset (data dictionary), keeping only vars available across all survey datasets
## stacking these and sorting by variable name allow the data user to compare the variable description acrosss time-points to ensure they are consistent, or alert the data-user to inconsistencies in variable description/definition across time-points 
#all_meta = pd.concat([meta1, meta2, meta3, meta4], keys=['feb2020','apr2020','jun2020','oct2020'], join="inner").reset_index().rename(columns={"level_0": "time-point"})
#all_meta = all_meta[all_meta['name'].isin(all_df.columns)].sort_values(by = ['name']) 

In [ ]:
# narrow down the dataset to only a few interesting (and relatively clean, straightforward variables) - check for missingness and impute to fill in missing

# get subset of interesting variables
#sub_df_1 = all_df[['time-point','weight','stigma_scale_score','age4','region4','personaluse_ever']]
sub_df_1 = df[['p_over','weight1','weight2','stigma_scale_score','expanded_10item_stigma','state','age4','racethnicity','educ5','personaluse_ever','familyuse_ever','personalcrimjust_ever','familycrimjust_ever']]

In [ ]:
sub_df_1.head()

In [ ]:
# check for missing I

# check if missing values
print("missing values: ")
print(sub_df_1.isnull().sum())

# check if missing values
print("missing values: ")
print(sub_df_1.isna().sum())

# get all var types
#sub_df_1.info()
print("var info: ")
print(sub_df_1.info())

# summary of numeric vars (weight and stigma_scale_score)
print("summary of numeric vars: ")
print(sub_df_1.describe())



In [ ]:
# check for missing II

# summary of cat vars (p_over, state, age4, racethnicity, educ5, personaluse_ever, familyuse_ever, personalcrimjustice_ever)

print("summary of cat var p_over: ")
print(sub_df_1.p_over.value_counts(dropna=False))

print("summary of cat var state: ")
print(sub_df_1.state.value_counts(dropna=False))

print("summary of cat var age4: ")
print(sub_df_1.age4.value_counts(dropna=False))

# summary of cat vars (age4, region9, personaluse_ever)
print("summary of cat var racethnicity: ")
print(sub_df_1.racethnicity.value_counts(dropna=False))

print("summary of cat var educ5: ")
print(sub_df_1.educ5.value_counts(dropna=False))

# summary of cat vars (age4, region9, personaluse_ever)
print("summary of cat var personaluse_ever: ")
print(sub_df_1.personaluse_ever.value_counts(dropna=False))

print("summary of cat var familyuse_ever: ")
print(sub_df_1.familyuse_ever.value_counts(dropna=False))

print("summary of cat var personalcrimjust_ever: ")
print(sub_df_1.personalcrimjust_ever.value_counts(dropna=False))

In [ ]:
# convert familycrimjust_ever var from 1/0 to yes/no to be in line with other covar format

#df.replace({0: 10, 1: 100},inplace=True)
sub_df_1.familycrimjust_ever.replace({0:"No",1:"Yes"},inplace=True)
print(sub_df_1.familycrimjust_ever.value_counts(dropna=False))


In [ ]:

# familyuse_ever var was coded as yes/' no' (space before 'no') instead of yes/no; convert it to be yes/no to be in line with other covar format

print(sub_df_1.familyuse_ever.value_counts(dropna=False))
sub_df_1.familyuse_ever.replace({" No":"No"},inplace=True)
print(sub_df_1.familyuse_ever.value_counts(dropna=False))


In [ ]:
# personalcrimjust_ever was coded as Yes, ever arrested or incarcerated/No, never arrested or incarcerated; convert it to be yes/no to be in line with other covar format

print(sub_df_1.personalcrimjust_ever.value_counts(dropna=False))
#sub_df_1.personalcrimjust_ever.replace(regex = {r'^Yes.$': "Yes", r'^No.$': "No"}, inplace=True)
sub_df_1.personalcrimjust_ever.replace({"Yes, ever arrested or incarcerated":"Yes", "No, never arrested or incarcerated":"No"},inplace=True)
print(sub_df_1.personalcrimjust_ever.value_counts(dropna=False))



In [ ]:
# impute missing stigma scale score vals with median per timepoint, impute missing personaluse_ever with mode, "No"

# impute missing stigma scale score values as the median score by survey time-point
#sub_df_1['stigma_scale_score'].fillna(sub_df_1.groupby('time-point')['stigma_scale_score'].transform('median'),inplace=True)
sub_df_1['stigma_scale_score'].fillna(sub_df_1['stigma_scale_score'].median(),inplace=True)
sub_df_1['expanded_10item_stigma'].fillna(sub_df_1['expanded_10item_stigma'].median(),inplace=True)
print(sub_df_1.isnull().sum())

# replace missing values of personaluse_ever with mode value of 'No'
sub_df_1.personaluse_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())

sub_df_1.familyuse_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())

sub_df_1.personalcrimjust_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())

sub_df_1.familycrimjust_ever.fillna('No',inplace=True)
print(sub_df_1.isnull().sum())



In [ ]:
sub_df_1.head()

In [ ]:
# get all var types
#sub_df_1.info()
print("var info: ")
print(sub_df_1.info())

# confirm missing values corrected
print("missing values: ")
print(sub_df_1.isnull().sum())

# confirm missing values corrected
print("missing values: ")
print(sub_df_1.isna().sum())

In [ ]:
# https://pythonfix.com/code/us-states-abbrev.py/

# state name to two letter code dictionary

us_state_to_abbrev = {
"Alabama": "AL",
"Alaska": "AK",
"Arizona": "AZ",
"Arkansas": "AR",
"California": "CA",
"Colorado": "CO",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Hawaii": "HI",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Iowa": "IA",
"Kansas": "KS",
"Kentucky": "KY",
"Louisiana": "LA",
"Maine": "ME",
"Maryland": "MD",
"Massachusetts": "MA",
"Michigan": "MI",
"Minnesota": "MN",
"Mississippi": "MS",
"Missouri": "MO",
"Montana": "MT",
"Nebraska": "NE",
"Nevada": "NV",
"New Hampshire": "NH",
"New Jersey": "NJ",
"New Mexico": "NM",
"New York": "NY",
"North Carolina": "NC",
"North Dakota": "ND",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pennsylvania": "PA",
"Rhode Island": "RI",
"South Carolina": "SC",
"South Dakota": "SD",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Washington": "WA",
"West Virginia": "WV",
"Wisconsin": "WI",
"Wyoming": "WY",
"District of Columbia": "DC",
"American Samoa": "AS",
"Guam": "GU",
"Northern Mariana Islands": "MP",
"Puerto Rico": "PR",
"United States Minor Outlying Islands": "UM",
"U.S. Virgin Islands": "VI",
}


In [ ]:
# add df column with state 2 letter code

state_cd = sub_df_1.state.replace(us_state_to_abbrev)
state_cd

sub_df_1.insert(6,"state_cd",state_cd,True)
sub_df_1.head()


In [ ]:
# define function to calculate weighted mean of across the full population for survey vars - necessary to properly weight in order to retain the validity of nationally representative survey sampling strategy 
# https://stackoverflow.com/questions/32771520/how-to-use-a-weighted-mean-estimator-in-seaborn-factor-plot-incl-bootstrapping

# sum of weights will not be equal to count of individuals when we look at sub-groups of the full population; 
# will have to formally calculate weighted average
def weighted_mean(x, **kws):
    val, weight = map(np.asarray, zip(*x))
    return (val * weight).sum() / weight.sum()

#sub_df_1["score_and_weight"] = list(zip(sub_df_1.stigma_scale_score, sub_df_1.weight))


In [ ]:
# https://github.com/mwaskom/seaborn/issues/722

def weighted_mean_2(x, **kws):
    return np.sum(np.real(x) * np.imag(x)) / np.sum(np.imag(x))


sub_df_1["score6_and_weight2"] = [ v + w*1j for v,w in zip(sub_df_1.stigma_scale_score, sub_df_1.weight2)]
sub_df_1["score10_and_weight2"] = [ v + w*1j for v,w in zip(sub_df_1.expanded_10item_stigma, sub_df_1.weight2)]


In [ ]:
def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [ ]:
# national estimate - method 1
print(weighted_mean_2(sub_df_1["score6_and_weight2"]))
print(weighted_mean_2(sub_df_1["score10_and_weight2"]))

# national estimate - method 2
print(wavg(sub_df_1,"stigma_scale_score","weight2"))
print(wavg(sub_df_1,"expanded_10item_stigma","weight2"))


In [ ]:
# state-level estimate - method 2 - all states

print(sub_df_1.groupby("state").apply(wavg,"stigma_scale_score","weight2"))
print(sub_df_1.groupby("state").apply(wavg,"expanded_10item_stigma","weight2"))


In [ ]:
# state-level estimate - method 2 - only oversampled states

# create new df with only state oversampled entries
sub_df_2.state.cat.remove_unused_categories(inplace = True)

# use new sub_df_2 to get state level estimates
print(sub_df_2.groupby("state").apply(wavg,"stigma_scale_score","weight2"))
print(sub_df_2.groupby("state").apply(wavg,"expanded_10item_stigma","weight2"))



In [ ]:

# state-level estimate - method 2 - only oversampled states
print(sub_df_1[sub_df_1.p_over == "AS oversample"].groupby("state", observed = True).apply(wavg,"stigma_scale_score","weight2"))
print(sub_df_1[sub_df_1.p_over == "AS oversample"].groupby("state", observed = True).apply(wavg,"expanded_10item_stigma","weight2"))

state_lvl_est_6 = sub_df_1[sub_df_1.p_over == "AS oversample"].groupby("state", observed = True).apply(wavg,"stigma_scale_score","weight2")
state_lvl_est_10 = sub_df_1[sub_df_1.p_over == "AS oversample"].groupby("state", observed = True).apply(wavg,"expanded_10item_stigma","weight2")


In [ ]:
state_lvl_est_6 = sub_df_1[sub_df_1.p_over == "AS oversample"].groupby("state", observed = True).apply(wavg,"stigma_scale_score","weight2")

state_lvl_est_6 = pd.DataFrame(state_lvl_est_6).reset_index().rename(columns={0:"stigma_scale_score"}).sort_values(by=["stigma_scale_score"],ascending=False)
state_lvl_est_6.head(10)

# add df column with state 2 letter code



def add_state_cd_to_df(df,state_name_col,insert_col,state_cd_col_name="state_cd"):
    # for df column with state full name, 
    # add df column with state 2 letter code
    # at desired location

    state_cd = df[state_name_col].replace(us_state_to_abbrev)
    state_cd

    # for now, leave in true to allow duplicates; should think about changing this though
    df.insert(insert_col,state_cd_col_name,state_cd,True)
    return df

state_lvl_est_6 = add_state_cd_to_df(df=state_lvl_est_6, state_name_col="state", insert_col=1)
state_lvl_est_6.head(10)    

In [ ]:
# quick map of state level estimates

fig = px.choropleth(state_lvl_est_6, locations="state_cd", locationmode="USA-states",scope="usa",color="stigma_scale_score",color_continuous_scale="Viridis_r")

fig.show()


In [ ]:
# iterating over a grouped df: http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#:~:text=the%20passed%20key.-,Iterating%20through%20groups,-%23

grouped_by_state = sub_df_2.groupby("state")

for name, group in grouped_by_state:
    print(name)
    print(group.weight2)

# https://allendowney.github.io/ElementsOfDataScience/12_bootstrap.html#weighted-bootstrapping

In [ ]:
from scipy.stats import bootstrap

# get a numpy array of stigma score values per state - need this to feed into scipy bootstrap function
state_stigma_value_arrays = sub_df_2.stigma_scale_score.groupby(sub_df_2.state).apply(np.array).values
print(state_stigma_value_arrays.ndim)
print(state_stigma_value_arrays.shape)
#print(state_stigma_value_arrays[5,])
print(state_stigma_value_arrays[14,].ndim)
print(state_stigma_value_arrays[14,].shape)

# get a numpy array of corresponding weight2 per state - need this to feed into scipy bootstrap function
state_stigma_value_arrays = sub_df_2.stigma_scale_score.groupby(sub_df_2.state).apply(np.array).values
print(state_stigma_value_arrays.ndim)
print(state_stigma_value_arrays.shape)
#print(state_stigma_value_arrays[5,])
print(state_stigma_value_arrays[14,].ndim)
print(state_stigma_value_arrays[14,].shape)




In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html

rng = np.random.default_rng()
from scipy.stats import norm
dist = norm(loc=2, scale=4) 
data = (dist.rvs(size=(n_trials, 100), random_state=rng),)
#res = bootstrap(data, np.std, axis=-1, confidence_level=0.9, n_resamples=1000, random_state=rng)
#ci_l, ci_u = res.confidence_interval
data

In [ ]:
# get all var types
#sub_df_1.info()
print("var info: ")
print(sub_df_1.info())

sub_df_1.head()

In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.5)

for i in ax.containers:
    ax.bar_label(i,)

In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.5, weights="weight2")

for i in ax.containers:
    ax.bar_label(i,)

In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.2, weights="weight2", hue="personaluse_ever", alpha = 0.6, stat="percent", common_norm=False, multiple="layer")
# setting common_norm == False gives percent within each group


#for i in ax.containers:
#    ax.bar_label(i,)


In [ ]:
ax = sns.histplot(x="stigma_scale_score", data=sub_df_1, binwidth=0.2, weights="weight2", hue="familyuse_ever", alpha = 0.6, stat="percent", common_norm=False, multiple="layer")
# setting common_norm == False gives percent within each group


In [ ]:
fig, ax = plt.subplots()

sns.barplot(ax=ax, x="state", y="score6_and_weight2", data=sub_df_1, estimator=weighted_mean_2, orient='v')
#ax.set_ylim(2, 3.55) 

plt.draw()
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,horizontalalignment='right')

lower = [line.get_ydata().min() for line in ax.lines]
upper = [line.get_ydata().max() for line in ax.lines]
#means = ax.collections[0].get_offsets()[:, 1]
#means = ax.containers
#means = ax.containers[0].get_offsets()[:, 1]
#means = ax.containers[0].get_datavalues()[:, 1]
means = [rectangle.get_height().max() for rectangle in ax.patches]
labels = [line.get_xdata() for line in ax.lines]
#print(lower)
#print(means)
print(labels)

In [ ]:
#https://stackoverflow.com/questions/54868292/extract-mean-and-confidence-intervals-from-seaborn-regplot

sns.barplot(x="state", y="score6_and_weight2", data=sub_df_1, estimator=weighted_mean_2, orient='v')
ax = plt.gca()
lower = [line.get_ydata().min() for line in ax.lines]
upper = [line.get_ydata().max() for line in ax.lines]
#means = ax.collections[0].get_offsets()[:, 1]
means = ax.containers
means


In [ ]:
fig, ax = plt.subplots()

sns.barplot(ax=ax, x="time-point", y="score_and_weight_2", data=sub_df_1, estimator=weighted_mean_2, orient='v', hue = "personaluse_ever")
#ax.set_ylim(2, 3.55) 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
fig.suptitle('Stigma Scale Score over time',fontsize='x-large',fontweight='bold')

one = sns.lineplot(ax = ax1,
             data = sub_df_1,
             x = 'time-point',
             y = 'score_and_weight_2',
             #estimator = 'mean'
             estimator = weighted_mean_2
             #hue = 'personaluse_ever'
            )

plt.draw()
one.set_xticks(one.get_xticks())
one.set_xticklabels(one.get_xticklabels(),rotation=45,horizontalalignment='right')
one.set_ylim(5, 1) # this is the average of 6 5-point likert scale questions; set the min/max accordingly
#for tick in one.get_xticklabels():
#    tick.set_rotation(45)
    
    
ax1.set_title('Stigma Scale Score')
ax1.set(ylabel='Stigma Scale Score')
ax1.set(xlabel='')
ax1.invert_yaxis()
#ax1.set_xticks(ax1.get_xticks())  # just get and reset whatever you already have
#one.set_xticklabels(one.get_xticklabels(),rotation=45,horizontalalignment='right')  # set the new/modified labels

two = sns.lineplot(ax = ax2,
             data = sub_df_1,
             x = 'time-point',
             y = 'score_and_weight_2',
             hue = 'personaluse_ever',
             #estimator = 'mean'
             estimator = weighted_mean_2
            )

plt.draw()
two.set_xticks(two.get_xticks())
two.set_xticklabels(two.get_xticklabels(),rotation=45,horizontalalignment='right')
two.set_ylim(5, 1) # this is the average of 6 5-point likert scale questions; set the min/max accordingly

ax2.set_title('Stigma Scale Score\nby Personal Use Status')
ax2.set(xlabel='')
ax2.invert_yaxis()

legend2 = ax2.legend()
#legend2.remove()

#leg = ax2.legend(loc='center', bbox_to_anchor=(1.1, -0.25), shadow=False, ncol=2, frameon=False)

plt.subplots_adjust(top=0.80)
plt.show()

```{margin} 
**To go to the data/study page on the HEAL Data Platform, follow this link:** my link
```

```{margin} 
**To go to an interactive analytic cloud workspace with the analysis code and data loaded, follow this link:** my link
```

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Sodales ut eu sem integer vitae justo eget. Pellentesque dignissim enim sit amet venenatis urna cursus. Sed faucibus turpis in eu mi bibendum. Scelerisque felis imperdiet proin fermentum leo. Volutpat est velit egestas dui id ornare arcu. Quis lectus nulla at volutpat diam ut venenatis tellus. Tellus pellentesque eu tincidunt tortor aliquam nulla facilisi cras. Pellentesque adipiscing commodo elit at imperdiet dui. 
<br>

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
fig.suptitle('Stigma Scale Score over time',fontsize='x-large',fontweight='bold')

one = sns.lineplot(ax = ax1,
             data = sub_df_1,
             x = 'time-point',
             y = 'score_and_weight_2',
             estimator = weighted_mean_2
             #hue = 'personaluse_ever'
            )

plt.draw()
one.set_xticks(one.get_xticks())
one.set_xticklabels(one.get_xticklabels(),rotation=45,horizontalalignment='right')
#for tick in one.get_xticklabels():
#    tick.set_rotation(45)
    
    
ax1.set_title('Stigma Scale Score')
ax1.set(ylabel='Stigma Scale Score')
ax1.set(xlabel='')
ax1.invert_yaxis()
#ax1.set_xticks(ax1.get_xticks())  # just get and reset whatever you already have
#one.set_xticklabels(one.get_xticklabels(),rotation=45,horizontalalignment='right')  # set the new/modified labels

two = sns.lineplot(ax = ax2,
             data = sub_df_1,
             x = 'time-point',
             y = 'score_and_weight_2',
             hue = 'personaluse_ever',
             estimator = weighted_mean_2
            )

plt.draw()
two.set_xticks(two.get_xticks())
two.set_xticklabels(two.get_xticklabels(),rotation=45,horizontalalignment='right')

ax2.set_title('Stigma Scale Score\nby Personal Use Status')
ax2.set(xlabel='')
ax2.invert_yaxis()

legend2 = ax2.legend()
#legend2.remove()

#leg = ax2.legend(loc='center', bbox_to_anchor=(1.1, -0.25), shadow=False, ncol=2, frameon=False)

plt.subplots_adjust(top=0.80)
plt.show()

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Sodales ut eu sem integer vitae justo eget. Pellentesque dignissim enim sit amet venenatis urna cursus. Sed faucibus turpis in eu mi bibendum. Scelerisque felis imperdiet proin fermentum leo. Volutpat est velit egestas dui id ornare arcu. Quis lectus nulla at volutpat diam ut venenatis tellus. Tellus pellentesque eu tincidunt tortor aliquam nulla facilisi cras. Pellentesque adipiscing commodo elit at imperdiet dui. 
<br><br>
In hac habitasse platea dictumst quisque sagittis purus. Libero volutpat sed cras ornare. Sit amet consectetur adipiscing elit pellentesque habitant morbi tristique senectus. Auctor augue mauris augue neque gravida in fermentum et. Amet mattis vulputate enim nulla aliquet porttitor. Proin sed libero enim sed faucibus turpis in eu. Morbi tristique senectus et netus et malesuada. Feugiat sed lectus vestibulum mattis ullamcorper.

**Data Citation** 
<br>
Harold Pollack, Johnathon Schneider, Bruce Taylor. JCOIN 026: Brief Stigma Survey. Chicago, IL: Center for Translational Data Science HEAL Data Platform (distributor) via Center for Translational Data Science JCOIN Data Commons (repository & distributor), 2022-04-08. (HEAL Data Platform branded doi goes here)
<br>
**Brief Article Citation** 
<br>
What format should this be? 